In [11]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

from create_staging_tables import create_staging_merged_initial_df

In [12]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')

In [13]:
con = engine.connect()

In [14]:
staging_vote_df = pd.read_sql_query('select * from "vote"',con=engine)

In [15]:
staging_bill_df = pd.read_sql_query('select * from "bill"',con=engine)

In [16]:
def create_staging_merged_initial_df(staging_vote_df, staging_bill_df):
    '''Create merged_initial by merging staging_bill_df and staging_vote_df on the bill_unique field.
    Input
    staging_bill_df: pandas dataframe loaded from wa_leg_staging database, bill table
    staging_vote_df:pandas dataframe loaded from wa_leg_staging database, vote table
    '''
    staging_vote_df = pd.read_sql_query('select * from "vote"',con=engine)
    staging_bill_df = pd.read_sql_query('select * from "bill"',con=engine)
    staging_bill_df['bill_unique'] = staging_bill_df['biennium'] + ' ' + staging_bill_df['bill_id']
    return staging_vote_df.merge(staging_bill_df, how='left', on='bill_unique')

In [17]:
MERGED_initial = create_staging_merged_initial_df(staging_vote_df, staging_bill_df)

In [18]:
MERGED_initial.head()

,sequence_number,vote,vote_date,voter_id,voter_name,voting_agency,bill_unique,year,biennium,bill_id,...,htm_last_modified_date,htm_url,long_friendly_name,name,type,bill_num,bill_num_unique,sponsor_agency,primary_sponsor_id,secondary_sponsors
0,38,1,1993-03-13,636,Flemming,0,1993-94 HB 1790,1993,1993-94,HB 1790,...,2006-07-15T03:59:45.197,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790,1790,House Bills,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22..."
1,38,1,1993-03-13,636,Flemming,0,1993-94 HB 1790,1993,1993-94,HB 1790,...,2006-07-15T07:24:34.477,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790 - as passed by the Legislature,1790.PL,House Passed Legislature,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22..."
2,38,1,1993-03-13,636,Flemming,0,1993-94 HB 1790,1993,1993-94,HB 1790,...,2012-08-12T14:25:39.837,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790 - Session Law,1790.SL,Session Laws,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22..."
3,38,1,1993-03-13,637,Foreman,0,1993-94 HB 1790,1993,1993-94,HB 1790,...,2006-07-15T03:59:45.197,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790,1790,House Bills,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22..."
4,38,1,1993-03-13,637,Foreman,0,1993-94 HB 1790,1993,1993-94,HB 1790,...,2006-07-15T07:24:34.477,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790 - as passed by the Legislature,1790.PL,House Passed Legislature,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22..."


In [ ]:
len(MERGED_initial)

In [ ]:
MERGED_initial[0:1].to_sql('merged_initial', con, if_exists='replace', index=False)

In [20]:
staging_vote_df.head()

,sequence_number,vote,vote_date,voter_id,voter_name,voting_agency,bill_unique,year
0,38,1,1993-03-13,636,Flemming,0,1993-94 HB 1790,1993
1,38,1,1993-03-13,637,Foreman,0,1993-94 HB 1790,1993
2,38,1,1993-03-13,140,Forner,0,1993-94 HB 1790,1993
3,38,1,1993-03-13,146,Fuhrman,0,1993-94 HB 1790,1993
4,38,1,1993-03-13,157,Grant,0,1993-94 HB 1790,1993


In [21]:
staging_bill_df['htm_create_date'] = pd.to_datetime(staging_bill_df['htm_create_date'])

In [22]:
staging_bill_df['htm_last_modified_date'] = pd.to_datetime(staging_bill_df['htm_last_modified_date'])

In [23]:
staging_bill_df[staging_bill_df['bill_unique'] == '1993-94 HB 1790']

,biennium,bill_id,class,description,htm_create_date,htm_last_modified_date,htm_url,long_friendly_name,name,type,bill_num,bill_num_unique,sponsor_agency,primary_sponsor_id,secondary_sponsors,bill_unique
8890,1993-94,HB 1790,Bills,None,1993-02-08,2006-07-15 03:59:45.197,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790,1790,House Bills,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22...",1993-94 HB 1790
8891,1993-94,HB 1790,Bills,3-29-93,1993-03-29,2006-07-15 07:24:34.477,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790 - as passed by the Legislature,1790.PL,House Passed Legislature,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22...",1993-94 HB 1790
8892,1993-94,HB 1790,Bills,CHAPTER 9 L 93,1993-04-12,2012-08-12 14:25:39.837,http://app.leg.wa.gov/documents/billdocs/1993-...,House Bill 1790 - Session Law,1790.SL,Session Laws,1790,1993-94 1790,House,973,"{645,652,219,207,17,642,650,449,657,622,234,22...",1993-94 HB 1790


In [24]:
150*19000

2850000

In [25]:
staging_bill_df['unique_id'] = np.linspace(1, len(staging_bill_df), len(staging_bill_df))

In [26]:
unique_vote_dates = staging_vote_df[['bill_unique', 'vote_date']]

In [27]:
unique_vote_dates.head()

,bill_unique,vote_date
0,1993-94 HB 1790,1993-03-13
1,1993-94 HB 1790,1993-03-13
2,1993-94 HB 1790,1993-03-13
3,1993-94 HB 1790,1993-03-13
4,1993-94 HB 1790,1993-03-13


In [28]:
unique_vote_dates.drop_duplicates(keep='first', inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
unique_vote_dates['unique_id'] = np.nan

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
unique_vote_dates = unique_vote_dates.reset_index()

In [37]:
unique_vote_dates.head()

,index,bill_unique,vote_date,unique_id
0,0,1993-94 HB 1790,1993-03-13,8893.0
1,70,1993-94 HB 1792,1993-04-18,NaN
2,79,1993-94 HB 1800,1993-04-18,NaN
3,120,1993-94 SHB 1792,1993-03-15,NaN
4,218,1993-94 SHB 1795,1993-03-12,NaN


In [49]:
def create_staging_unique_vote_dates_df(staging_bill_df, staging_vote_df):
    '''Identify the exact bill that the legislators voted on. Create a dataframe that conists of all vote_date 
    and bill_unique pairs. Create a null unique_id field that labels each bill with a unique ID, and begins as 
    null for unique_vote_dates. For each row in unique_vote_dates identify the bill that was created closest to, 
    but before, the vote date. Set the unique_id of that bill to the unique_id of that vote. This will be use to 
    later join the bill_df to the vote_df.'''
    
    unique_vote_dates = staging_vote_df[['bill_unique', 'vote_date']]
    unique_vote_dates.drop_duplicates(keep='first', inplace=True)
    unique_vote_dates = unique_vote_dates.reset_index()
    
    unique_vote_dates['unique_id'] = np.nan
    staging_bill_df['unique_id'] = np.linspace(1, len(staging_bill_df), len(staging_bill_df))
    
    count = 0
    for i1, row in unique_vote_dates.iterrows():
        time_diffs = {}
        bill_options = staging_bill_df[staging_bill_df['bill_unique'] == row['bill_unique']]
        for i2, option in bill_options.iterrows():
            time_diff = option['htm_create_date'] - row['vote_date']
            if time_diff <= pd.to_timedelta('0'):
                time_diffs[time_diff] = option['unique_id']
            if time_diff > pd.to_timedelta('0'):
                time_diff -= pd.to_timedelta('-1000 days')
                time_diffs[time_diff] = option['unique_id']
        if len(time_diffs) > 0:
            bill_voted_on = time_diffs[max(time_diffs)]
            unique_vote_dates.iloc[i1, -1] = bill_voted_on
    return unique_vote_dates

In [47]:
unique_vote_dates

,index,bill_unique,vote_date,unique_id
0,0,1993-94 HB 1790,1993-03-13,8893.0
1,70,1993-94 HB 1792,1993-04-18,8895.0
2,79,1993-94 HB 1800,1993-04-18,8909.0
3,120,1993-94 SHB 1792,1993-03-15,8896.0
4,218,1993-94 SHB 1795,1993-03-12,8900.0
5,316,1993-94 SHB 1799,1993-03-13,8906.0
6,462,1993-94 HB 1800,1993-04-17,8909.0
7,560,1993-94 SHB 1801,1993-04-12,8913.0
8,609,1993-94 SHB 1801,1993-03-12,8913.0
9,658,1993-94 HB 1809,1993-03-13,8935.0


In [48]:
unique_vote_dates.to_sql('unique_vote_dates', con, if_exists='replace', index=False)

In [45]:
len(x)

2